## Object individualization (Chap 13)

#### Singletons

In [1]:
# as shown earlier - you can define singleton methods
# directly on individual objects
obj = Object.new
def obj.talk
    puts "howdy."
end
obj.talk

howdy.


In [2]:
# most common type of singleton method = the class method
# - a method added to a class object, individual basis.
class Car
    def self.makes
        %w{honda ford toyota chevy volvo}
    end
end

:makes

In [27]:
# directly examining & modifying a singleton class
# singleton classes = anonymous, but you can add instance methods, class methods, constant.
# to do this:
# class << object denotes anon singleton class of object.
#   defs go here
# end

str = "i am a string"
class << str
    def twice # singleton method of the string str
        self + " " + self
    end
end
puts str.twice

# class<<object notation usually for class-method definitions

i am a string i am a string


Singleton classes in method lookup chain:
![chain](px/Selection_174.png)

In [29]:
# including a module in a singleton class
class Person
    attr_accessor :name
end

d = Person.new; d.name = "david"
j = Person.new; j.name = "joe"
r = Person.new; r.name = "ruby"

module Secretive
    def name
        "anonymous"
    end
end

class << r
    include Secretive
end

puts "#{d.name}"; puts "#{j.name}"; puts "#{r.name}"

david
joe
anonymous


In [30]:
# singleton_class method - refers directly to singleton class of an object
str = "a string"; puts str.singleton_class.ancestors

[#<Class:#<String:0x000055764659d988>>, ActiveSupport::ToJsonWithActiveSupportEncoder, String, JSON::Ext::Generator::GeneratorMethods::String, Comparable, ActiveSupport::ToJsonWithActiveSupportEncoder, Object, ActiveSupport::Tryable, PP::ObjectMixin, JSON::Ext::Generator::GeneratorMethods::Object, Kernel, BasicObject]


In [33]:
# class methods, advanced
# singleton methods defined on an object == no other object can be a receiver
# singleton methods defined on a class == can be called on its subclasses too.

class C 
end
def C.a_class_method
    puts "singleton method defined on C."
end
C.a_class_method

class D<C
end
D.a_class_method

# singleton classes of class objects sometimes called "meta classes".

singleton method defined on C.
singleton method defined on C.


#### Modifying Ruby core classes & modules

In [11]:
# risks - changes to Ruby core classes are GLOBAL.
# example candidate for ad-hoc changes: Regexp
# default: match fail ==> nil; match success ==> MatchData object
# nil has no [] method, so [1] ==> NoMethodError
# first try:

class Regexp
    alias __oldmatch__ match
    def match(str)
        __oldmatch__(str) || []
    end
end

# test it out:
/abc/.match("X")[1] # failure won't blow up

# problem is, empty arrays interpreted as true. screws up dependent code.

In [14]:
# another example of risk: String#gsub!
str = "hello there"
print(str.gsub!(/e/,"E")+"\n")
print(str)

hEllo thErE
hEllo thErE

In [16]:
# but when gsub! doesn't make any changes: gsub! return value is nil...
str = "hello there"
print(str.gsub!(/zzz/,"xxx"))
print(str)

hello there

In [17]:
# tap - executes a code block, yields receiver to block, returns receiver.
"Hello".tap {|str| puts str.upcase}.reverse

HELLO


"olleH"

In [18]:
# changing core functionality safely: technique #1 (additive changes)
# (adding a method that doesn't exist)
# good example: Active Support
require 'active_support'
require 'active_support/core_ext'
print("person".pluralize)
print("little_dorritt".titleize)

peopleLittle Dorritt

In [19]:
# changing core functionality safely: technique #1 (pass-through overrides)
# original version of method gets called along with new version
# good for logging & debugging
class String
    alias __oldreverse__ reverse
    def reverse
        print("reversing a string!")
        __oldreverse__
    end
end
print("david".reverse)

reversing a string!divad

In [20]:
# changing core functionality safely: technique #1a (additive/pass-through)
# Active Support does this - example is to_s
print(Time.now.to_s)
print(Time.now.to_s(:db))

2020-09-21 19:42:15 -05002020-09-21 19:42:15

In [23]:
# changing core functionality safely: technique #2 (per-object changes with extend)
module Secretive
    def name
        "not available"
    end
end
class Person
    attr_accessor :name
end

# extend adds Secretive to lookup paths of individual objects by mixing it into 
# their singleton classes.

david = Person.new; david.name = "david"; david.extend(Secretive)
joe   = Person.new;   joe.name = "joe"
ruby  = Person.new;  ruby.name = "ruby";   ruby.extend(Secretive)
puts "#{david.name}"
puts "#{joe.name}"
puts "#{ruby.name}"


not available
joe
not available


In [24]:
# changing core functionality safely: technique #3 (refinements)
# make a temporary, limited-scope change
module Shout
    refine String do 
        def shout
            self.upcase+"!!!"
        end
    end
end

# refine: takes class name & code block that defines "refined" behaviors.
# using: target class adopts the new behaviors.

class Person
    attr_accessor :name
    using Shout
    def announce
        puts "announcing #{name.shout}"
    end
end
david = Person.new; david.name = "David"; david.announce

announcing DAVID!!!


#### BasicObject
- enables you to create "do nothing" objects - you can teach them to do anything
- typically this means heavy use of _method_missing_.
- best example: _Builder_ library for XML writing.

In [9]:
# BasicObject = top of Ruby class tree.
String.ancestors.last

BasicObject

In [10]:
# BO is a class & behaves like one.
BasicObject.instance_methods(false).sort

[:!, :!=, :==, :__binding__, :__id__, :__send__, :equal?, :instance_eval, :instance_exec]

In [6]:
# usage (output *should* be XML; NEEDS DEBUG IN JUPYTER CONTEXT.)
require 'builder'
xml = Builder::XmlMarkup.new(target: STDOUT, indent: 2)
xml.instruct! # tells XML to start with XML declaration
xml.friends do 
    xml.friend(source: "career") do 
        xml.name("joe")
        xml.address do 
            xml.street("123 main st")
            xml.city("anywhere usa 00000")
        end
    end
end

#<IO:<STDOUT>>

In [5]:
# subclass of BasicObject
# Lister -- redefines method_missing
class Lister<BasicObject
    attr_reader :list
    def initialize
        @list = ""; @level = 0
    end
    def indent(str)
        " " * @level + str.to_s
    end
    def method_missing(m, &block)
        @list  << indent(m) + ":" + "\n"
        @level += 2
        @list  << indent(yield(self)) if block 
        @level -= 2
        @list  << "\n"
        return ""
    end
end

lister = Lister.new
lister.groceries do |item|
    item.name { "apples "}
    item.qty  { 10       }
    item.name { "sugar"  }
    item.qty  { "1 lb"   }
end
lister.freeze do |f|
    f.name { "ice cream" }
end
lister.inspect do |i|
    i.itm { "car" }
end
lister.sleep do |s|
    s.hours { 8 }
end
lister.print do |doc|
    doc.book { "ch 13" }
    doc.letter { "to editor" }
end
puts lister.list

groceries:
  name:
    apples 
  qty:
    10
  name:
    sugar
  qty:
    1 lb
  
freeze:
  name:
    ice cream
  
inspect:
  itm:
    car
  
sleep:
  hours:
    8
  
print:
  book:
    ch 13
  letter:
    to editor
  

